## Develop summary table of all Sensors in LoRa Database

In [16]:
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone

pd.options.display.max_rows = 200

In [17]:
gtw_file = '~/gateways.tsv'
df = pd.read_csv(gtw_file, sep='\t', parse_dates=['ts'])
df.head()

,dev_id,ts,ts_day,ts_hour,counter,gateway,snr,rssi,data_rate
0,ldds75-a84041281182a87f,2021-09-22 00:00:50,2021-09-22,2021-09-22 00:00,79,packetbroker,9.50,-72,SF7BW125.0
1,erslite-marquam-56a5,2021-09-22 00:00:57,2021-09-22,2021-09-22 00:00,40,packetbroker,10.25,-31,SF7BW125.0
2,ersco2-msb-admin-26db,2021-09-22 00:01:04,2021-09-22,2021-09-22 00:00,19959,msb-admin-a840411eb4f84150,10.50,-30,SF7BW125.0
3,ersco2-msb-admin-26db,2021-09-22 00:01:04,2021-09-22,2021-09-22 00:00,19959,packetbroker,4.00,-107,SF7BW125.0
4,ersco2-msb-admin-26db,2021-09-22 00:01:04,2021-09-22,2021-09-22 00:00,19959,packetbroker,-6.20,-122,SF7BW125.0


In [18]:
tz_ak = timezone('US/Alaska')
ts_start = (datetime.now(tz_ak) - timedelta(days=1)).replace(tzinfo=None)
ts_start

datetime.datetime(2021, 10, 21, 19, 50, 22, 537641)

In [19]:
df1d = df.query('ts >= @ts_start')

In [20]:
dfs = df1d.groupby('dev_id').agg(
    {
        'data_rate': 'last',
        'counter': ['first', 'last']
    }
)
dfs.columns = ['data_rate', 'counter_first', 'counter_last']
dfs['total_rdg'] = dfs.counter_last - dfs.counter_first + 1
dfs.head()

,data_rate,counter_first,counter_last,total_rdg
dev_id,,,,
boat-lt2-a725,SF10BW125.0,5411,5435,25
boat-lt2-a726,SF10BW125.0,2101,2374,274
boat-lt2-a728,SF10BW125.0,299,32,-266
boat-lt2-a72b,SF10BW125.0,9304,9576,273
boat-lt2-a72d,SF10BW125.0,3575,3847,273


In [21]:
df_rcvd = df1d[['dev_id', 'counter']].drop_duplicates()
df_rcvd = df_rcvd.groupby('dev_id').count()
df_rcvd.columns = ['rcvd']
df_rcvd.head()

,rcvd
dev_id,
boat-lt2-a725,2
boat-lt2-a726,266
boat-lt2-a728,274
boat-lt2-a72b,267
boat-lt2-a72d,260


In [23]:
df_final = dfs.join(df_rcvd)
df_final['success_pct'] = df_final.rcvd / df_final.total_rdg
df_display = df_final[['data_rate', 'success_pct']].copy()
df_display['data_rate'] = df_display.data_rate.str.replace('.0', '', regex=False)
df_display.columns = ['Data Rate', 'Success %']
df_display.index.name = 'Dev ID'
s2 = df_display.style.applymap(lambda v: 'color:red;' if v < 0.9 else None, subset=['Success %'])
s2

,Data Rate,Success %
Dev ID,,
boat-lt2-a725,SF10BW125,0.080000
boat-lt2-a726,SF10BW125,0.970803
boat-lt2-a728,SF10BW125,-1.030075
boat-lt2-a72b,SF10BW125,0.978022
boat-lt2-a72d,SF10BW125,0.952381
boat-lt2-a72f,SF10BW125,-0.848708
boat-lt2-a84041552182436a,SF10BW125,0.970696
boat-lt2-a840417f8182436e,SF10BW125,0.956044
boat-lt2-phoenix-swd,SF7BW125,0.956044
